Instalando pacotes

Transformers

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 5.0 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 636 kB 78.6 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.0 MB/s 


Dataset

In [ ]:
! pip install datasets

     |████████████████████████████████| 270 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 61.7 MB/s 
     |████████████████████████████████| 125 kB 73.3 MB/s 
     |████████████████████████████████| 1.3 MB 60.7 MB/s 
     |████████████████████████████████| 294 kB 63.2 MB/s 
     |████████████████████████████████| 160 kB 73.7 MB/s 


In [ ]:
! pip install simpletransformers

     |████████████████████████████████| 231 kB 5.0 MB/s 
     |████████████████████████████████| 1.7 MB 65.1 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 8.3 MB 25.9 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 180 kB 65.3 MB/s 
     |████████████████████████████████| 139 kB 72.5 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 111 kB 73.5 MB/s 
     |████████████████████████████████| 76 kB 6.6 MB/s 
     |████████████████████████████████| 4.3 MB 30.3 MB/s 
     |████████████████████████████████| 124 kB 72.3 MB/s 
     |████████████████████████████████| 788 kB 75.8 MB/s 
     |████████████████████████████████| 370 kB 77.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=a18750c2e787e0a5dbe71108bd52099f2e7bc325d6f44630a4591fe3921d9ac2
  Stored in directory: /root/.cache/

Googletranslation

In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 10.3 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 65 kB 4.8 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=8d65cf5dddd365e822fd7d4e232b1d8656863f096016aad0fe87b39563d1505b
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 3.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=d2b36d6a2196236c917a856f470cbef1f89fe85fe594a75d854ca49cb556dee1
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
!pip install datasets

In [ ]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 4.9 MB/s 


Baixando o Dataset

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

In [ ]:
dataset = load_dataset('tapaco', 'pt')

Reusing dataset tapaco (/root/.cache/huggingface/datasets/tapaco/pt/1.0.0/71d200534b520a174927a8f0479c06220a0a6fb5201a84ebfce19006c6354698)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tapaco = []
# The dataset has only train split.
for data in tqdm(dataset["train"]):
    keys = data.keys()
    tapaco.append([data[key] for key in keys])


100%|██████████| 78430/78430 [00:07<00:00, 10618.42it/s]


In [ ]:
tapaco_df = pd.DataFrame(
    data=tapaco,
    columns=[
       "paraphrase_set_id",
       "sentence_id",
        "paraphrase",
        "lists",
        "tags",
        "language",
    ],
)

In [ ]:
tapaco_df.head()

,paraphrase_set_id,sentence_id,paraphrase,lists,tags,language
0,1,421918,Eu comi o queijo.,[],[],pt
1,1,5045315,Eu estou comendo queijo.,[],[],pt
2,1,5045316,Estou comendo queijo.,[],[],pt
3,5,620684,Hoje é segunda-feira.,[],[OK],pt
4,5,1227813,É segunda.,[4138],[],pt


In [ ]:
def generate_tapaco_paraphrase_dataset(dataset):
    dataset_df = dataset[["paraphrase", "paraphrase_set_id"]]
    non_single_labels = (
        dataset_df["paraphrase_set_id"]
        .value_counts()[dataset_df["paraphrase_set_id"].value_counts() > 1]
        .index.tolist()
    )
    tapaco_df_sorted = dataset_df.loc[
        dataset_df["paraphrase_set_id"].isin(non_single_labels)
    ]
    tapaco_paraphrases_dataset = []

    for paraphrase_set_id in tqdm(tapaco_df_sorted["paraphrase_set_id"].unique()):
        id_wise_paraphrases = tapaco_df_sorted[
            tapaco_df_sorted["paraphrase_set_id"] == paraphrase_set_id
        ]
        len_id_wise_paraphrases = (
            id_wise_paraphrases.shape[0]
            if id_wise_paraphrases.shape[0] % 2 == 0
            else id_wise_paraphrases.shape[0] - 1
        )
        for ix in range(0, len_id_wise_paraphrases, 2):
            current_phrase = id_wise_paraphrases.iloc[ix][0]
            for count_ix in range(ix + 1, ix + 2):
                next_phrase = id_wise_paraphrases.iloc[ix + 1][0]
                tapaco_paraphrases_dataset.append([current_phrase, next_phrase])
    tapaco_paraphrases_dataset_df = pd.DataFrame(
        tapaco_paraphrases_dataset, columns=["Text", "Paraphrase"]
    )
    
    return tapaco_paraphrases_dataset_df

In [ ]:
dataset_df = generate_tapaco_paraphrase_dataset(tapaco_df)
dataset_df.head()

100%|██████████| 29949/29949 [02:40<00:00, 186.14it/s]


,Text,Paraphrase
0,Eu comi o queijo.,Eu estou comendo queijo.
1,Hoje é segunda-feira.,É segunda.
2,É segunda-feira.,Hoje é segunda.
3,Onde se encontra a biblioteca?,Onde é a biblioteca?
4,Você fala inglês?,Você não fala inglês?


In [ ]:
from simpletransformers.t5 import T5Model
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
dataset_df.columns = ["input_text","target_text"]
dataset_df["prefix"] = "parafrase"

In [ ]:
dataset_df.shape

(36451, 3)

In [ ]:
dataset_df.head()

,input_text,target_text,prefix
0,Eu comi o queijo.,Eu estou comendo queijo.,parafrase
1,Hoje é segunda-feira.,É segunda.,parafrase
2,É segunda-feira.,Hoje é segunda.,parafrase
3,Onde se encontra a biblioteca?,Onde é a biblioteca?,parafrase
4,Você fala inglês?,Você não fala inglês?,parafrase


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
original_embedding = model.encode(dataset_df['input_text'])
dataset_df['original_embedding'] = [original_embedding[i] for i in range(original_embedding.shape[0])]


parafrase_embedding = model.encode(dataset_df['target_text'])
dataset_df['parafrase_embedding'] = [parafrase_embedding[i] for i in range(parafrase_embedding.shape[0])]

dataset_df['similiaridade_parafrase'] = dataset_df.progress_apply(lambda x:util.cos_sim(x['original_embedding'], x['parafrase_embedding']).tolist()[0][0], axis=1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
100%|██████████| 36451/36451 [00:03<00:00, 10360.07it/s]


In [ ]:
dataset_df.head()

,input_text,target_text,prefix,original_embedding,parafrase_embedding,similiaridade_parafrase
0,Eu comi o queijo.,Eu estou comendo queijo.,parafrase,"[-0.085935086, 0.04360156, -0.003897462, -0.05...","[-0.05510287, -0.027534671, -0.0024288862, -0....",0.882763
1,Hoje é segunda-feira.,É segunda.,parafrase,"[-0.06608112, -0.17827009, -0.011088491, -0.09...","[-0.04838354, -0.1305625, -0.014201537, -0.070...",0.958786
2,É segunda-feira.,Hoje é segunda.,parafrase,"[-0.07405615, -0.15982755, -0.010273912, -0.08...","[-0.060619485, -0.17244618, -0.012098313, -0.0...",0.956231
3,Onde se encontra a biblioteca?,Onde é a biblioteca?,parafrase,"[0.09929005, 0.29744226, -0.006044928, 0.05992...","[0.11998385, 0.3395866, -0.0058360873, 0.04559...",0.991387
4,Você fala inglês?,Você não fala inglês?,parafrase,"[-0.224415, 0.18841593, -0.012384672, -0.03341...","[-0.17771542, 0.25026986, -0.013931634, -0.019...",0.924461


In [ ]:
dataset_df['similiaridade_parafrase'].describe()

count    36451.000000
mean         0.920216
std          0.119332
min         -0.136561
25%          0.896255
50%          0.970207
75%          0.994300
max          0.999963
Name: similiaridade_parafrase, dtype: float64

In [ ]:
fim = dataset_df[dataset_df['similiaridade_parafrase'] >= 0.8].copy()
fim = fim [["input_text","target_text", "prefix"]].copy()

In [ ]:
train_data,test_data = train_test_split(dataset_df,test_size=0.1)

In [ ]:
train_data.shape

(32805, 6)

In [ ]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.85,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "evaluate_during_training": False,
    'adam_epsilon': 1e-08,
    'eval_batch_size': 6,
    'fp_16': True,
    'gradient_accumulation_steps': 16,
    'learning_rate': 0.0003,
    'max_grad_norm': 1.0,
    'n_gpu': 1,
    'seed': 42,
    'train_batch_size': 6,
    'warmup_steps': 0,
    'weight_decay': 0.0
}

In [ ]:
model = T5Model("t5",'unicamp-dl/ptt5-base-portuguese-vocab', args=args)

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/738k [00:00<?, ?B/s]

In [ ]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/32805 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3399: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Using Adafactor for T5


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/5468 [00:00<?, ?it/s]

(1364, 1.2167284926208408)

In [ ]:
prefix = "parafrase"
pred = model.predict([f"{prefix}: Adoro treinar modelos no python."])
print(pred)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['Eu gosto de treinar modelos no python.']
